In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Настройка стиля графиков
plt.style.use('default')
sns.set_palette("husl")

# Чтение файла
df = pd.read_csv('bs140513_032310.csv')

labels=['zipcodeOri', 'zipMerchant']
df=df.drop(labels, axis=1, inplace=False, errors='raise')



# Убедимся, что данные отсортированы по времени (важно для "до текущего дня")
df = df.sort_values(['customer', 'step']).reset_index(drop=True)

# 1. Общее количество транзакций за всё время для клиента
df['cust_total_tx'] = df.groupby('customer')['step'].transform('count')

# 2. Количество уникальных мерчантов, с которыми клиент взаимодействовал (всё время)
df['cust_unique_merch_total'] = df.groupby('customer')['merchant'].transform('nunique')

# 3. Сколько транзакций у клиента до текущего дня (исключая текущую)
# Для этого используем cumulative count - 1
df['cust_tx_before'] = df.groupby('customer').cumcount()

# 4. Сколько уникальных мерчантов у клиента до текущего дня
# Нужно аккуратно: считаем уникальные merchant_id до текущей строки
def count_unique_before(series):
    # series — это merchant_id для одного клиента, отсортированный по day
    seen = set()
    result = []
    for merch in series:
        result.append(len(seen))
        seen.add(merch)
    return result

df['cust_unique_merch_before'] = df.groupby('customer')['merchant'].transform(count_unique_before)

# 5. Бинарный признак: первая ли это транзакция клиента?
df['is_first_tx'] = (df['cust_tx_before'] == 0).astype(int)

# 6. Взаимодействовал ли клиент с этим мерчантом раньше? (до текущей транзакции)
def has_seen_merchant(group):
    seen = set()
    res = []
    for merch in group['merchant']:
        res.append(1 if merch in seen else 0)
        seen.add(merch)
    return res

df['seen_merchant_before'] = df.groupby('customer').apply(has_seen_merchant).explode().values.astype(int)

# 7. Доля дней с транзакциями (всё время)
# Сначала посчитаем уникальные дни у клиента
cust_unique_days = df.groupby('customer')['step'].nunique()
df['cust_unique_days'] = df['customer'].map(cust_unique_days)
df['cust_tx_days_ratio'] = df['cust_unique_days'] / 180.0  # 180 — общее число дней

# 8. Максимальное число транзакций за один день (всё время)
daily_counts = df.groupby(['customer', 'step']).size().reset_index(name='daily_tx')
max_daily = daily_counts.groupby('customer')['daily_tx'].max()
df['cust_max_tx_per_day'] = df['customer'].map(max_daily)

# 9. Средний интервал между транзакциями (всё время)
def avg_interval(days):
    if len(days) <= 1:
        return np.nan  # или 0, но лучше NaN
    diffs = np.diff(sorted(days))
    return diffs.mean()

cust_avg_int = df.groupby('customer')['step'].apply(avg_interval)
df['cust_avg_interval'] = df['customer'].map(cust_avg_int)

# 10. Среднее число транзакций в день у клиента (до текущего дня)
# = cust_tx_before / (текущий день - первый день активности + 1), но проще:
# Будем считать как: (число транзакций до текущего дня) / (число дней от первой транзакции до текущего дня)
def avg_tx_per_day_before(group):
    days = group['step'].values
    tx_counts = np.arange(len(days))  # 0, 1, 2, ..., n-1
    first_day = days[0]
    result = []
    for i, day in enumerate(days):
        if i == 0:
            result.append(0.0)  # или np.nan
        else:
            span_days = day - first_day
            if span_days == 0:
                result.append(np.nan)
            else:
                result.append(tx_counts[i] / span_days)
    return result

df['cust_avg_tx_per_day_before'] = df.groupby('customer').apply(
    lambda g: pd.Series(avg_tx_per_day_before(g), index=g.index)
).values

# 11. Количество транзакций за последние 7 дней (до текущего дня, не включая текущую)
# Используем rolling window по day (но day — не обязательно последовательные индексы)
# Поэтому делаем через цикл или merge с временным окном

# Создадим вспомогательный датафрейм для rolling-подсчёта
df_sorted = df.copy()
df_sorted = df_sorted.sort_values(['customer', 'step'])

# Инициализируем колонку
df_sorted['tx_last_7_days'] = 0

# Группируем по клиенту и считаем для каждой транзакции
for cust_id, group in df_sorted.groupby('customer'):
    days = group['step'].values
    indices = group.index
    counts = []
    for i, current_day in enumerate(days):
        # Считаем транзакции в диапазоне (current_day - 7, current_day)
        start_day = current_day - 7
        # Только предыдущие транзакции (строго < current_day)
        count = np.sum((days[:i] > start_day) & (days[:i] < current_day))
        counts.append(count)
    df_sorted.loc[indices, 'tx_last_7_days'] = counts

# Вернём результат в исходный порядок (если был)
df = df_sorted.sort_index().reset_index(drop=True)



# Обработка категориальных признаков
labels = ['step', 'customer', 'merchant']
df = df.drop(labels, axis=1, inplace=False, errors='raise')

# Применяем one-hot encoding к столбцам 'gender' и 'category'
df_encoded = pd.get_dummies(df, columns=['gender', 'category'], prefix=['gender', 'category'])

# Теперь преобразуем булевы значения в 0/1 и очищаем age сразу после создания df_encoded
df_encoded = df_encoded.replace({True: 1, False: 0})
df_encoded['age'] = pd.to_numeric(df_encoded['age'].str.replace("'", ""), errors='coerce')

# Теперь df_encoded содержит исходные столбцы (кроме 'gender' и 'category')
# и новые бинарные столбцы вида gender_M, gender_F, category_food, category_travel и т.д.

# print(df_encoded.head())
# print("\nСтолбцы после обработки:")
# print(df_encoded.columns)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
from scipy.cluster import hierarchy
import scipy.cluster.hierarchy
import scipy.spatial.distance
%matplotlib inline

# Сохраняем fraud отдельно для будущего анализа
fraud_labels = df_encoded['fraud']  # сохраняем целевую переменную

# Определяем признаки для кластеризации (все кроме fraud)
features = [col for col in df_encoded.columns if col != 'fraud' and col != 'cust_unique_days']
data = df_encoded[features]

data.columns

"""## 3. Стандартизация данных"""

# Разделяем непрерывные и бинарные признаки
continuous_features = ['age', 'amount', 'cust_total_tx', 'cust_unique_merch_total',
                      'cust_tx_before', 'cust_unique_merch_before',
                      'cust_max_tx_per_day', 'cust_avg_interval', 'cust_avg_tx_per_day_before', 'cust_tx_days_ratio', 'tx_last_7_days']

# Все остальные признаки считаем бинарными (one-hot encoded)
binary_features = [col for col in features if col not in continuous_features]

print(f"Непрерывные признаки ({len(continuous_features)}): {continuous_features}")
print(f"Бинарные признаки ({len(binary_features)}): {binary_features[:10]}...")  # покажем первые 10

# Стандартизируем только непрерывные признаки
scaled_continuous = (data[continuous_features] - data[continuous_features].mean(axis=0)) / data[continuous_features].std()
scaled_continuous = scaled_continuous.fillna(0)

# Бинарные признаки оставляем как есть (0/1)
scaled_binary = data[binary_features]

# Объединяем
scaled = pd.concat([scaled_continuous, scaled_binary], axis=1)

print("\nДанные после стандартизации:")
print(scaled.head())
print(f"\nДиапазоны значений:")
print("Непрерывные:", scaled[continuous_features].describe().loc[['min', 'max']].T)
print("Бинарные: 0 или 1")

/tmp/ipython-input-86643222.py:56: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['seen_merchant_before'] = df.groupby('customer').apply(has_seen_merchant).explode().values.astype(int)
/tmp/ipython-input-86643222.py:98: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['cust_avg_tx_per_day_before'] = df.groupby('customer').apply(
/tmp/ipython-input-86643222.py:139: FutureWarning: Downcasting behavior in `r

Непрерывные признаки (11): ['age', 'amount', 'cust_total_tx', 'cust_unique_merch_total', 'cust_tx_before', 'cust_unique_merch_before', 'cust_max_tx_per_day', 'cust_avg_interval', 'cust_avg_tx_per_day_before', 'cust_tx_days_ratio', 'tx_last_7_days']
Бинарные признаки (21): ['is_first_tx', 'seen_merchant_before', "gender_'E'", "gender_'F'", "gender_'M'", "gender_'U'", "category_'es_barsandrestaurants'", "category_'es_contents'", "category_'es_fashion'", "category_'es_food'"]...

Данные после стандартизации:
        age    amount  cust_total_tx  cust_unique_merch_total  cust_tx_before  \
0  1.504337  0.951321      -0.918952                 0.865256       -1.617505   
1  1.504337 -0.190302      -0.918952                 0.865256       -1.596803   
2  1.504337  0.164178      -0.918952                 0.865256       -1.576101   
3  1.504337 -0.207806      -0.918952                 0.865256       -1.555400   
4  1.504337  0.085544      -0.918952                 0.865256       -1.534698   

  

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 10.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from catboost import CatBoostClassifier
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
import time
from scipy.stats import randint, uniform
import joblib

class LargeDataBoostingOptimizer:
    def __init__(self, n_jobs=-1, random_state=42):
        self.n_jobs = n_jobs
        self.random_state = random_state
        self.best_models = {}
        self.results = {}

    def create_optimized_models(self):
        """Создаем модели с оптимизированными параметрами для больших данных"""

        models = {
            'AdaBoost': AdaBoostClassifier(
                n_estimators=200,  # Увеличиваем для больших данных
                learning_rate=0.5,  # Более агрессивное обучение
                random_state=self.random_state
            ),

            'Gradient Boosting': GradientBoostingClassifier(
                n_estimators=200,
                learning_rate=0.05,  # Меньше learning rate для стабильности
                max_depth=6,  # Глубже для сложных паттернов
                subsample=0.8,  # Субсемплинг для ускорения
                max_features='sqrt',  # Уменьшение переобучения
                random_state=self.random_state,
                validation_fraction=0.1,  # Валидация для ранней остановки
                n_iter_no_change=10
            ),

            'CatBoost': CatBoostClassifier(
                iterations=1000,  # Больше итераций
                learning_rate=0.05,
                depth=8,  # Глубже
                l2_leaf_reg=3,  # Регуляризация
                random_seed=self.random_state,
                verbose=False,
                early_stopping_rounds=50,  # Ранняя остановка
                thread_count=self.n_jobs,
                used_ram_limit='4gb'  # Контроль памяти
            ),

            'Stochastic Gradient Boosting': HistGradientBoostingClassifier(
                max_iter=200,
                learning_rate=0.05,
                max_depth=8,
                min_samples_leaf=20,  # Предотвращение переобучения
                l2_regularization=1.0,  # Регуляризация
                max_bins=255,  # Для производительности
                early_stopping=True,
                validation_fraction=0.1,
                n_iter_no_change=10,
                random_state=self.random_state
            ),

            'XGBoost': xgb.XGBClassifier(
                n_estimators=1000,  # Больше деревьев с ранней остановкой
                learning_rate=0.05,
                max_depth=8,
                subsample=0.8,
                colsample_bytree=0.8,
                reg_alpha=1,  # L1 регуляризация
                reg_lambda=1,  # L2 регуляризация
                random_state=self.random_state,
                n_jobs=self.n_jobs,
                tree_method='hist'  # Для больших данных
            )
        }

        return models

    def optimize_for_large_data(self, X, y, sample_size=100000):
        """Быстрая оптимизация на подвыборке"""
        print(f"Быстрая оптимизация на подвыборке {sample_size} примеров...")

        # Берем подвыборку для быстрой оптимизации
        if len(X) > sample_size:
            X_sample, _, y_sample, _ = train_test_split(
                X, y, train_size=sample_size, random_state=self.random_state, stratify=y
            )
        else:
            X_sample, y_sample = X, y

        param_distributions = {
            'XGBoost': {
                'learning_rate': uniform(0.01, 0.1),
                'max_depth': randint(4, 10),
                'subsample': uniform(0.7, 0.3),
                'colsample_bytree': uniform(0.7, 0.3),
                'reg_alpha': uniform(0, 2),
                'reg_lambda': uniform(0, 2)
            },
            'CatBoost': {
                'learning_rate': uniform(0.01, 0.1),
                'depth': randint(4, 10),
                'l2_leaf_reg': uniform(1, 4)
            }
        }

        optimized_params = {}

        # Оптимизируем только ключевые модели
        for model_name in ['XGBoost', 'CatBoost']:
            print(f"Оптимизация {model_name}...")

            if model_name == 'XGBoost':
                model = xgb.XGBClassifier(
                    n_estimators=100,
                    random_state=self.random_state,
                    n_jobs=self.n_jobs,
                    tree_method='hist'
                )
            else:
                model = CatBoostClassifier(
                    iterations=100,
                    random_seed=self.random_state,
                    verbose=False,
                    thread_count=self.n_jobs
                )

            search = RandomizedSearchCV(
                model,
                param_distributions[model_name],
                n_iter=10,  # Малое количество итераций для скорости
                cv=3,
                scoring='f1',
                random_state=self.random_state,
                n_jobs=1  # Чтобы не перегружать память
            )

            search.fit(X_sample, y_sample)
            optimized_params[model_name] = search.best_params_
            print(f"Лучшие параметры для {model_name}: {search.best_params_}")

        return optimized_params

    def train_with_early_stopping(self, X_train, X_val, y_train, y_val, models):
        """Обучение с ранней остановкой для экономии времени"""
        trained_models = {}

        for name, model in models.items():
            print(f"Обучение {name} с ранней остановкой...")
            start_time = time.time()

            if name == 'CatBoost':
                model.fit(
                    X_train, y_train,
                    eval_set=(X_val, y_val),
                    early_stopping_rounds=50,
                    verbose=False
                )
            elif name == 'XGBoost':
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_val, y_val)],
                    verbose=False
                )
            elif name in ['Gradient Boosting', 'Stochastic Gradient Boosting']:
                # Эти модели уже имеют встроенную раннюю остановку
                model.fit(X_train, y_train)
            else:
                model.fit(X_train, y_train)

            training_time = time.time() - start_time
            print(f"{name} обучен за {training_time:.2f} секунд")
            trained_models[name] = model

        return trained_models

    def evaluate_model(self, y_true, y_pred, y_pred_proba=None):
        metrics = {
            'Accuracy': accuracy_score(y_true, y_pred),
            'Precision': precision_score(y_true, y_pred, zero_division=0),
            'Recall': recall_score(y_true, y_pred, zero_division=0),
            'F1': f1_score(y_true, y_pred, zero_division=0)
        }

        if y_pred_proba is not None:
            metrics['AUC'] = roc_auc_score(y_true, y_pred_proba)
        else:
            metrics['AUC'] = roc_auc_score(y_true, y_pred)

        return metrics

    def compare_boosting_methods(self, X, y, test_size=0.2, val_size=0.1):
        """Основная функция сравнения для больших данных"""

        # Разделяем данные
        X_temp, X_test, y_temp, y_test = train_test_split(
            X, y, test_size=test_size, random_state=self.random_state, stratify=y
        )

        # Дополнительное разделение для валидации (ранняя остановка)
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp, y_temp, test_size=val_size/(1-test_size),
            random_state=self.random_state, stratify=y_temp
        )

        print(f"Размеры данных: Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

        # Оптимизация параметров на подвыборке
        optimized_params = self.optimize_for_large_data(X_train, y_train)

        # Создаем модели
        base_models = self.create_optimized_models()

        # Применяем оптимизированные параметры
        for model_name, params in optimized_params.items():
            if model_name == 'XGBoost':
                base_models[model_name].set_params(**params)
            elif model_name == 'CatBoost':
                base_models[model_name].set_params(**params)

        # Обучение с ранней остановкой
        trained_models = self.train_with_early_stopping(X_train, X_val, y_train, y_val, base_models)

        # Оценка на тестовой выборке
        results = {}
        for name, model in trained_models.items():
            print(f"Оценка {name}...")

            y_pred = model.predict(X_test)

            if hasattr(model, 'predict_proba'):
                y_pred_proba = model.predict_proba(X_test)[:, 1]
            else:
                y_pred_proba = None

            metrics = self.evaluate_model(y_test, y_pred, y_pred_proba)
            results[name] = metrics

            # Сохраняем лучшие модели
            self.best_models[name] = model

        self.results = results
        return results, trained_models

    def print_results(self):
        """Вывод результатов"""
        results_df = pd.DataFrame(self.results).T
        results_df = results_df.round(4)

        print("\n" + "="*100)
        print("СРАВНЕНИЕ МЕТОДОВ БУСТИНГА ДЛЯ БОЛЬШИХ ДАННЫХ")
        print("="*100)
        print(results_df.to_string())

        print("\n" + "="*100)
        print("ЛУЧШИЕ МОДЕЛИ ПО МЕТРИКАМ:")
        print("="*100)

        for metric in ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']:
            best_model = results_df[metric].idxmax()
            best_score = results_df[metric].max()
            print(f"{metric}: {best_model} - {best_score:.4f}")

    def save_best_model(self, model_name, filepath):
        """Сохранение лучшей модели"""
        if model_name in self.best_models:
            joblib.dump(self.best_models[model_name], filepath)
            print(f"Модель {model_name} сохранена в {filepath}")

    def load_and_predict(self, model_path, X_new):
        """Загрузка модели и предсказание"""
        model = joblib.load(model_path)
        return model.predict(X_new)

# Пример использования с большими данными
def main_large_data():
    # Создаем большую синтетическую выборку для демонстрации

    X, y = scaled, fraud_labels

    # Инициализируем оптимизатор
    optimizer = LargeDataBoostingOptimizer(n_jobs=4, random_state=42)

    # Сравниваем методы
    print("Запуск сравнения методов бустинга...")
    results, models = optimizer.compare_boosting_methods(X, y)

    # Выводим результаты
    optimizer.print_results()

    # Анализ производительности
    print("\n" + "="*100)
    print("РЕКОМЕНДАЦИИ ДЛЯ БОЛЬШИХ ДАННЫХ:")
    print("="*100)

    results_df = pd.DataFrame(results).T
    best_overall = results_df['F1'].idxmax()

    print(f"Рекомендуемая модель: {best_overall}")
    print("Причины:")

    if best_overall in ['XGBoost', 'CatBoost']:
        print("- Оптимизированы для распределенных вычислений")
        print("- Эффективное использование памяти")
        print("- Встроенные механизмы регуляризации")
    elif best_overall == 'Stochastic Gradient Boosting':
        print("- Высокая скорость на больших данных")
        print("- Хорошая регуляризация")

    # Сохраняем лучшую модель
    optimizer.save_best_model(best_overall, f'best_{best_overall}_model.pkl')

# # Дополнительная функция для инкрементального обучения
# def incremental_learning_example():
#     """Пример инкрементального обучения для очень больших данных"""
#     from sklearn.ensemble import HistGradientBoostingClassifier

#     # Создаем генератор данных (в реальности загружаем пакетами)
#     def data_generator(batch_size=50000, total_samples=600000):
#         for i in range(0, total_samples, batch_size):
#             X_batch, y_batch = make_classification(
#                 n_samples=batch_size,
#                 n_features=30,
#                 random_state=42 + i
#             )
#             yield X_batch, y_batch

#     # Инкрементальное обучение
#     model = HistGradientBoostingClassifier(
#         max_iter=100,
#         learning_rate=0.1,
#         early_stopping=False,  # Отключаем для инкрементального обучения
#         random_state=42
#     )

#     print("Инкрементальное обучение...")
#     for i, (X_batch, y_batch) in enumerate(data_generator()):
#         print(f"Пакет {i + 1}...")
#         if i == 0:
#             model.fit(X_batch, y_batch)
#         else:
#             # Додобучение на новых данных
#             model.set_params(warm_start=True)
#             model.fit(X_batch, y_batch)

if __name__ == "__main__":
    from sklearn.datasets import make_classification
    main_large_data()

Запуск сравнения методов бустинга...
Размеры данных: Train: (416249, 32), Val: (59465, 32), Test: (118929, 32)
Быстрая оптимизация на подвыборке 100000 примеров...
Оптимизация XGBoost...
Лучшие параметры для XGBoost: {'colsample_bytree': np.float64(0.7093939877366675), 'learning_rate': np.float64(0.09422847745949985), 'max_depth': 7, 'reg_alpha': np.float64(1.8789978831283782), 'reg_lambda': np.float64(1.7896547008552977), 'subsample': np.float64(0.8793699936433255)}
Оптимизация CatBoost...
Лучшие параметры для CatBoost: {'depth': 6, 'l2_leaf_reg': np.float64(1.0823379771832098), 'learning_rate': np.float64(0.10699098521619943)}
Обучение AdaBoost с ранней остановкой...
AdaBoost обучен за 99.23 секунд
Обучение Gradient Boosting с ранней остановкой...
Gradient Boosting обучен за 75.76 секунд
Обучение CatBoost с ранней остановкой...
CatBoost обучен за 43.49 секунд
Обучение Stochastic Gradient Boosting с ранней остановкой...
Stochastic Gradient Boosting обучен за 18.51 секунд
Обучение XGBo

# Неудачные попытки

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
import warnings
warnings.filterwarnings('ignore')

class ComBoost(BaseEstimator, ClassifierMixin):
    """
    ComBoost (Комитетный бустинг) - алгоритм построения компактных композиций

    Параметры:
    -----------
    base_estimator : object, default=None
        Базовый классификатор
    n_estimators : int, default=10
        Максимальное количество базовых алгоритмов
    l1_ratio : float, default=0.1
        Доля объектов, считающихся выбросами (нижняя граница)
    l2_ratio : float, default=0.9
        Доля объектов для обучения (верхняя граница)
    optimize_params : bool, default=True
        Оптимизировать ли параметры l1, l2 на каждом шаге
    early_stopping_rounds : int, default=5
        Количество раундов без улучшения для остановки
    random_state : int, default=None
        Seed для воспроизводимости
    """

    def __init__(self, base_estimator=None, n_estimators=10, l1_ratio=0.1,
                 l2_ratio=0.9, optimize_params=True, early_stopping_rounds=5,
                 random_state=None):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.l1_ratio = l1_ratio
        self.l2_ratio = l2_ratio
        self.optimize_params = optimize_params
        self.early_stopping_rounds = early_stopping_rounds
        self.random_state = random_state

        if self.base_estimator is None:
            self.base_estimator = SVC(kernel='linear', probability=True, random_state=random_state)

    def fit(self, X, y):
        """Обучение комитетного бустинга"""
        X, y = check_X_y(X, y)
        self.classes_ = unique_labels(y)
        self.X_, self.y_ = X, y
        self.n_samples_, self.n_features_ = X.shape

        # Инициализация
        self.estimators_ = []  # список базовых алгоритмов
        self.margins_history_ = []  # история отступов
        self.errors_history_ = []  # история ошибок

        # Преобразование меток в {-1, +1}
        self.y_binary_ = np.where(y == self.classes_[0], -1, 1)

        # Начальные отступы
        current_margins = np.zeros(self.n_samples_)

        best_error = float('inf')
        no_improvement_count = 0

        print("ComBoost training started...")

        for t in range(self.n_estimators):
            # Упорядочивание объектов по возрастанию отступов
            sorted_indices = np.argsort(current_margins)

            if self.optimize_params:
                # Оптимизация параметров l1, l2 на каждом шаге
                l1_best, l2_best, error_best = self._optimize_parameters(
                    X, self.y_binary_, current_margins, sorted_indices
                )
            else:
                # Фиксированные параметры
                l1_best = int(self.l1_ratio * self.n_samples_)
                l2_best = int(self.l2_ratio * self.n_samples_)

            # Обучение нового базового алгоритма на подвыборке
            start_idx = l1_best
            end_idx = l2_best

            if end_idx <= start_idx:
                end_idx = start_idx + 1

            train_indices = sorted_indices[start_idx:end_idx]

            if len(np.unique(self.y_binary_[train_indices])) < 2:
                # Пропускаем если в подвыборке только один класс
                continue

            estimator = self._clone_estimator()
            estimator.fit(X[train_indices], self.y_binary_[train_indices])

            # Добавление алгоритма в композицию
            self.estimators_.append(estimator)

            # Обновление отступов
            predictions = estimator.predict(X)
            current_margins += predictions
            self.margins_history_.append(current_margins.copy())

            # Вычисление ошибки
            current_error = np.sum(current_margins * self.y_binary_ < 0)
            self.errors_history_.append(current_error)

            print(f"Step {t+1}: Error = {current_error}/{self.n_samples_}, "
                  f"l1={l1_best}, l2={l2_best}, estimators={len(self.estimators_)}")

            # Проверка улучшения
            if current_error < best_error:
                best_error = current_error
                no_improvement_count = 0
            else:
                no_improvement_count += 1

            # Удаление старых алгоритмов (если нужно)
            if len(self.estimators_) > 3 and no_improvement_count > 2:
                # Удаляем самый старый алгоритм
                removed_estimator = self.estimators_.pop(0)
                # Пересчитываем отступы
                current_margins -= removed_estimator.predict(X)
                print(f"  Removed oldest estimator. Total estimators: {len(self.estimators_)}")

            # Критерий останова
            if (no_improvement_count >= self.early_stopping_rounds or
                current_error == 0):
                break

        self.n_estimators_ = len(self.estimators_)
        print(f"Training completed. Final committee size: {self.n_estimators_}")

        return self

    def _optimize_parameters(self, X, y, current_margins, sorted_indices):
        """Оптимизация параметров l1, l2"""
        best_error = float('inf')
        best_l1, best_l2 = 0, self.n_samples_

        # Поиск по сетке параметров
        l1_options = [0, int(0.05 * self.n_samples_), int(0.1 * self.n_samples_)]
        l2_options = [int(0.7 * self.n_samples_), int(0.8 * self.n_samples_),
                     int(0.9 * self.n_samples_), self.n_samples_]

        for l1 in l1_options:
            for l2 in l2_options:
                if l2 <= l1:
                    continue

                train_indices = sorted_indices[l1:l2]

                if len(np.unique(y[train_indices])) < 2:
                    continue

                # Быстрое обучение на подвыборке
                temp_estimator = self._clone_estimator()
                try:
                    temp_estimator.fit(X[train_indices], y[train_indices])

                    # Оценка качества на всей выборке
                    temp_predictions = temp_estimator.predict(X)
                    temp_margins = current_margins + temp_predictions
                    temp_error = np.sum(temp_margins * y < 0)

                    if temp_error < best_error:
                        best_error = temp_error
                        best_l1, best_l2 = l1, l2
                except:
                    continue

        return best_l1, best_l2, best_error

    def _clone_estimator(self):
        """Создание копии базового классификатора"""
        import copy
        return copy.deepcopy(self.base_estimator)

    def predict(self, X):
        """Предсказание классов"""
        check_is_fitted(self)
        X = check_array(X)

        if len(self.estimators_) == 0:
            return np.ones(X.shape[0]) * self.classes_[0]

        # Голосование комитета
        scores = self.decision_function(X)
        binary_predictions = np.sign(scores)

        # Преобразование обратно в исходные метки
        return np.where(binary_predictions == -1, self.classes_[0], self.classes_[1])

    def decision_function(self, X):
        """Функция принятия решений (взвешенная сумма прогнозов)"""
        check_is_fitted(self)
        X = check_array(X)

        if len(self.estimators_) == 0:
            return np.zeros(X.shape[0])

        # Сумма прогнозов всех алгоритмов комитета
        scores = np.zeros(X.shape[0])
        for estimator in self.estimators_:
            try:
                predictions = estimator.predict(X)
                scores += predictions
            except:
                # Если predict недоступен, используем predict_proba
                probas = estimator.predict_proba(X)
                if probas.shape[1] == 2:
                    predictions = 2 * probas[:, 1] - 1
                else:
                    predictions = probas[:, 1] - probas[:, 0]
                scores += predictions

        return scores

    def get_margins(self, X=None, y=None):
        """Получение отступов для объектов"""
        check_is_fitted(self)

        if X is None:
            X = self.X_
        if y is None:
            y = self.y_binary_

        scores = self.decision_function(X)
        return y * scores

    def get_outlier_indices(self, threshold_ratio=0.1):
        """Получение индексов объектов-выбросов"""
        check_is_fitted(self)

        margins = self.get_margins()
        n_outliers = int(threshold_ratio * len(margins))

        # Выбросы - объекты с наименьшими отступами
        outlier_indices = np.argsort(margins)[:n_outliers]
        return outlier_indices

# Сравнительный класс для экспериментов
class ComBoostComparator:
    """Класс для сравнения ComBoost с другими методами"""

    def __init__(self, random_state=42):
        self.random_state = random_state

    def evaluate_methods(self, X, y, test_size=0.2):
        """Сравнение различных методов бустинга"""
        from sklearn.model_selection import train_test_split
        from sklearn.ensemble import AdaBoostClassifier
        from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

        # Разделение данных
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=self.random_state, stratify=y
        )

        methods = {}
        results = {}

        # ComBoost с оптимизацией параметров
        print("=== Training ComBoost with optimization ===")
        comboost_opt = ComBoost(
            base_estimator=SVC(kernel='linear', probability=True, random_state=self.random_state),
            n_estimators=20,
            optimize_params=True,
            random_state=self.random_state
        )
        comboost_opt.fit(X_train, y_train)
        methods['ComBoost_opt'] = comboost_opt

        # ComBoost без оптимизации параметров
        print("\n=== Training ComBoost without optimization ===")
        comboost_fixed = ComBoost(
            base_estimator=SVC(kernel='linear', probability=True, random_state=self.random_state),
            n_estimators=20,
            optimize_params=False,
            random_state=self.random_state
        )
        comboost_fixed.fit(X_train, y_train)
        methods['ComBoost_fixed'] = comboost_fixed

        # AdaBoost для сравнения
        print("\n=== Training AdaBoost ===")
        adaboost = AdaBoostClassifier(
            estimator=SVC(kernel='linear', probability=True, random_state=self.random_state),
            n_estimators=50,
            random_state=self.random_state
        )
        adaboost.fit(X_train, y_train)
        methods['AdaBoost'] = adaboost

        # Одиночный SVM для сравнения
        print("\n=== Training Single SVM ===")
        single_svm = SVC(kernel='linear', probability=True, random_state=self.random_state)
        single_svm.fit(X_train, y_train)
        methods['Single_SVM'] = single_svm

        # Оценка всех методов
        for name, model in methods.items():
            y_pred = model.predict(X_test)

            results[name] = {
                'Accuracy': accuracy_score(y_test, y_pred),
                'Precision': precision_score(y_test, y_pred, average='weighted', zero_division=0),
                'Recall': recall_score(y_test, y_pred, average='weighted', zero_division=0),
                'F1': f1_score(y_test, y_pred, average='weighted', zero_division=0),
                'N_Estimators': len(model.estimators_) if hasattr(model, 'estimators_') else 1
            }

            if hasattr(model, 'estimators_'):
                print(f"{name}: {len(model.estimators_)} estimators")

        return results, methods

# Пример использования
if __name__ == "__main__":
    from sklearn.datasets import load_breast_cancer
    from sklearn.preprocessing import StandardScaler
    import pandas as pd

    # Загрузка данных
    X, y = scaled, fraud_labels


    # Сравнение методов
    print("Starting comparison...")
    comparator = ComBoostComparator(random_state=42)
    results, models = comparator.evaluate_methods(X, y)

    # Вывод результатов
    print("\n" + "="*60)
    print("COMPARISON RESULTS")
    print("="*60)

    results_df = pd.DataFrame(results).T
    print(results_df.round(4))

    # Анализ ComBoost
    comboost_model = models['ComBoost_opt']
    print(f"\nComBoost analysis:")
    print(f"Final committee size: {comboost_model.n_estimators_}")
    print(f"Training error progression: {comboost_model.errors_history_}")

    # Выбросы
    outliers = comboost_model.get_outlier_indices(threshold_ratio=0.1)
    print(f"Detected outliers (first 10): {outliers[:10]}")

Starting comparison...
=== Training ComBoost with optimization ===
ComBoost training started...


In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from typing import List, Tuple, Callable
import warnings

class BoostingAlgorithm:
    def __init__(self, l0: int, l1: int, l2: int, delta_l: int,
                 max_iter: int = 10, min_improvement: float = 0.01,
                 random_state: int = 42):
        """
        Инициализация алгоритма

        Parameters:
        l0, l1, l2: параметры отсечения для индексов
        delta_l: шаг для перебора k
        max_iter: максимальное количество итераций
        min_improvement: минимальное улучшение для продолжения алгоритма
        random_state: для воспроизводимости
        """
        self.l0 = l0
        self.l1 = l1
        self.l2 = l2
        self.delta_l = delta_l
        self.max_iter = max_iter
        self.min_improvement = min_improvement
        self.random_state = random_state

        self.base_algorithms = []
        self.margins_history = []
        self.Q_history = []

    def _create_base_algorithms(self):
        """Создание базовых алгоритмов"""
        return [
            ('SVM', SVC(kernel='linear', probability=True, random_state=self.random_state)),
            ('DecisionTree', DecisionTreeClassifier(random_state=self.random_state)),
            ('LogisticRegression', LogisticRegression(random_state=self.random_state))
        ]

    def _calculate_margin(self, X: np.ndarray, y: np.ndarray,
                         algorithms: List[Tuple[str, object]]) -> np.ndarray:
        """
        Расчет отступа для объектов

        Parameters:
        X: признаки
        y: метки
        algorithms: список базовых алгоритмов

        Returns:
        margins: отступы для каждого объекта
        """
        margins = np.zeros(len(X))

        for name, algo in algorithms:
            # Получаем вероятности принадлежности классу +1
            if hasattr(algo, 'predict_proba'):
                probas = algo.predict_proba(X)
                # Берем вероятность класса +1 (второй столбец)
                if probas.shape[1] == 2:
                    scores = probas[:, 1]
                else:
                    scores = probas[:, 0]
            else:
                scores = algo.decision_function(X)

            margins += y * scores

        return margins

    def _calculate_Q(self, margins: np.ndarray) -> int:
        """
        Расчет функционала качества композиции

        Parameters:
        margins: отступы объектов

        Returns:
        Q: количество объектов с отступом < 0
        """
        return np.sum(margins < 0)

    def _find_best_algorithm(self, X: np.ndarray, y: np.ndarray,
                            current_algorithms: List[Tuple[str, object]]) -> Tuple[object, str, float]:
        """
        Поиск лучшего базового алгоритма

        Parameters:
        X: признаки
        y: метки
        current_algorithms: текущий список алгоритмов

        Returns:
        best_algo: лучший алгоритм
        best_name: название лучшего алгоритма
        best_Q: лучшее значение Q
        """
        base_algos = self._create_base_algorithms()
        best_Q = float('inf')
        best_algo = None
        best_name = None

        for name, algo_class in base_algos:
            try:
                # Обучаем алгоритм
                algo = algo_class.fit(X, y)

                # Временно добавляем к текущим алгоритмам
                temp_algorithms = current_algorithms + [(name, algo)]

                # Рассчитываем отступы
                margins = self._calculate_margin(X, y, temp_algorithms)

                # Рассчитываем Q
                Q = self._calculate_Q(margins)

                if Q < best_Q:
                    best_Q = Q
                    best_algo = algo
                    best_name = name

            except Exception as e:
                warnings.warn(f"Алгоритм {name} вызвал ошибку: {e}")
                continue

        return best_algo, best_name, best_Q

    def fit(self, X: np.ndarray, y: np.ndarray,
            sample_size: float = None, stratify: bool = True) -> 'BoostingAlgorithm':
        """
        Обучение алгоритма

        Parameters:
        X: признаки
        y: метки (-1 или +1)
        sample_size: доля выборки для уменьшения (None - использовать всю)
        stratify: использовать стратификацию при уменьшении выборки

        Returns:
        self: обученный алгоритм
        """
        # Уменьшение выборки если нужно
        if sample_size is not None and sample_size < 1.0:
            if stratify:
                X, _, y, _ = train_test_split(
                    X, y,
                    train_size=sample_size,
                    stratify=y,
                    random_state=self.random_state
                )
            else:
                indices = np.random.choice(
                    len(X),
                    size=int(len(X) * sample_size),
                    replace=False
                )
                X = X[indices]
                y = y[indices]

        # Проверка меток
        unique_labels = np.unique(y)
        if not (set(unique_labels) == {-1, 1} or set(unique_labels) == {0, 1}):
            raise ValueError("Метки должны быть -1/+1 или 0/1")

        # Преобразование меток 0/1 в -1/+1 если нужно
        if set(unique_labels) == {0, 1}:
            y = np.where(y == 0, -1, 1)

        # Шаг 1: построение первого базового алгоритма
        print("Шаг 1: Построение первого базового алгоритма")
        best_algo, best_name, best_Q = self._find_best_algorithm(X, y, [])

        self.base_algorithms = [(best_name, best_algo)]
        current_margins = self._calculate_margin(X, y, self.base_algorithms)

        # Сортировка по отступам
        sorted_indices = np.argsort(current_margins)
        X_sorted = X[sorted_indices]
        y_sorted = y[sorted_indices]
        margins_sorted = current_margins[sorted_indices]

        self.Q_history.append(best_Q)
        self.margins_history.append(current_margins.copy())

        print(f"Первый алгоритм: {best_name}, Q = {best_Q}")

        # Основной цикл
        for t in range(self.max_iter):
            print(f"\nИтерация {t + 1}")

            best_iter_algo = None
            best_iter_name = None
            best_iter_Q = float('inf')
            best_k = None

            # Вложенный цикл по k
            for k in range(self.l1, self.l2 + 1, self.delta_l):
                if k <= self.l0 or k >= len(X_sorted):
                    continue

                # Выбираем подвыборку от l0 до k
                sub_indices = range(self.l0, k)
                X_sub = X_sorted[sub_indices]
                y_sub = y_sorted[sub_indices]

                # Строим базовый алгоритм
                algo, name, Q = self._find_best_algorithm(X_sub, y_sub, self.base_algorithms)

                if Q < best_iter_Q and algo is not None:
                    best_iter_Q = Q
                    best_iter_algo = algo
                    best_iter_name = name
                    best_k = k

            # Проверяем улучшение
            if best_iter_algo is None:
                print("Не удалось найти подходящий алгоритм на этой итерации")
                break

            # Добавляем лучший алгоритм в композицию
            self.base_algorithms.append((best_iter_name, best_iter_algo))

            # Пересчитываем отступы
            current_margins = self._calculate_margin(X, y, self.base_algorithms)

            # Сортировка по отступам
            sorted_indices = np.argsort(current_margins)
            X_sorted = X[sorted_indices]
            y_sorted = y[sorted_indices]
            margins_sorted = current_margins[sorted_indices]

            # Сохраняем историю
            self.Q_history.append(best_iter_Q)
            self.margins_history.append(current_margins.copy())

            print(f"Добавлен алгоритм: {best_iter_name}, k = {best_k}, Q = {best_iter_Q}")

            # Проверяем условие остановки
            if len(self.Q_history) >= 2:
                improvement = self.Q_history[-2] - self.Q_history[-1]
                if improvement < self.min_improvement:
                    print(f"Улучшение ({improvement}) меньше минимального ({self.min_improvement}), остановка")
                    break

        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Предсказание для новых данных

        Parameters:
        X: признаки

        Returns:
        predictions: предсказанные метки (-1 или +1)
        """
        if not self.base_algorithms:
            raise ValueError("Модель не обучена")

        margins = self._calculate_margin(X, np.ones(len(X)), self.base_algorithms)
        return np.where(margins >= 0, 1, -1)

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """
        Вероятности принадлежности классу +1

        Parameters:
        X: признаки

        Returns:
        probabilities: вероятности класса +1
        """
        if not self.base_algorithms:
            raise ValueError("Модель не обучена")

        margins = self._calculate_margin(X, np.ones(len(X)), self.base_algorithms)
        # Преобразуем отступы в вероятности с помощью сигмоиды
        probabilities = 1 / (1 + np.exp(-margins))
        return probabilities

    def get_algorithm_info(self) -> List[Tuple[str, float]]:
        """
        Получение информации об алгоритмах в композиции

        Returns:
        info: список кортежей (название алгоритма, точность на обучающей выборке)
        """
        info = []
        for i, (name, algo) in enumerate(self.base_algorithms):
            info.append((f"{name}_{i}", name))
        return info

# Пример использования
if __name__ == "__main__":
    from sklearn.datasets import make_classification
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report

    # Генерация тестовых данных
    X, y = scaled, fraud_labels

    # Преобразование меток в -1/+1
    y = np.where(y == 0, -1, 1)

    # Разделение на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    # Инициализация и обучение алгоритма
    booster = BoostingAlgorithm(
        l0=6000,      # начальный индекс отсечения
        l1=45000,      # начальное значение k
        l2=55000,     # конечное значение k
        delta_l=2000, # шаг для k
        max_iter=10,
        min_improvement=1,
        random_state=42
    )

    # Обучение с уменьшением выборки (80%) и стратификацией
    booster.fit(X_train, y_train, sample_size=0.1, stratify=True)

    # Предсказание
    y_pred = booster.predict(X_test)

    # Оценка качества
    print("\n" + "="*50)
    print("Результаты классификации:")
    print(classification_report(y_test, y_pred, target_names=['Class -1', 'Class +1']))

    print(f"Точность: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Финальное значение Q: {booster.Q_history[-1]}")
    print(f"Количество алгоритмов в композиции: {len(booster.base_algorithms)}")

    # История значений Q
    print("\nИстория значений Q:")
    for i, q in enumerate(booster.Q_history):
        print(f"Итерация {i}: Q = {q}")

Шаг 1: Построение первого базового алгоритма


KeyError: "None of [Index([27673, 27663, 27664, 27665, 27666, 27667, 27668, 27669, 27670, 27671,\n       ...\n       16150, 26657,  2164, 39428, 11187, 40838, 13616, 16133, 18561, 19935],\n      dtype='int64', length=41625)] are in the [columns]"

In [ ]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from typing import List, Tuple, Callable
import warnings

class BoostingAlgorithm:
    def __init__(self, l0: int, l1: int, l2: int, delta_l: int,
                 max_iter: int = 10, min_improvement: float = 0.01,
                 random_state: int = 42):
        """
        Инициализация алгоритма

        Parameters:
        l0, l1, l2: параметры отсечения для индексов
        delta_l: шаг для перебора k
        max_iter: максимальное количество итераций
        min_improvement: минимальное улучшение для продолжения алгоритма
        random_state: для воспроизводимости
        """
        self.l0 = l0
        self.l1 = l1
        self.l2 = l2
        self.delta_l = delta_l
        self.max_iter = max_iter
        self.min_improvement = min_improvement
        self.random_state = random_state

        self.base_algorithms = []
        self.margins_history = []
        self.Q_history = []

    def _create_base_algorithms(self):
        """Создание базовых алгоритмов"""
        return [
            ('SVM', SVC(kernel='linear', probability=True, random_state=self.random_state)),
            ('DecisionTree', DecisionTreeClassifier(random_state=self.random_state, max_depth=3)),
            ('LogisticRegression', LogisticRegression(random_state=self.random_state))
        ]

    def _ensure_numpy(self, X, y):
        """Преобразование в numpy массивы и сброс индексов"""
        if hasattr(X, 'values'):
            X = X.values
        if hasattr(y, 'values'):
            y = y.values
        return X, y

    def _calculate_margin(self, X: np.ndarray, y: np.ndarray,
                         algorithms: List[Tuple[str, object]]) -> np.ndarray:
        """
        Расчет отступа для объектов

        Parameters:
        X: признаки
        y: метки
        algorithms: список базовых алгоритмов

        Returns:
        margins: отступы для каждого объекта
        """
        margins = np.zeros(len(X))

        for name, algo in algorithms:
            try:
                # Получаем вероятности принадлежности классу +1
                if hasattr(algo, 'predict_proba'):
                    probas = algo.predict_proba(X)
                    # Берем вероятность класса +1 (второй столбец)
                    if probas.shape[1] == 2:
                        scores = probas[:, 1]
                    else:
                        scores = probas[:, 0]
                else:
                    scores = algo.decision_function(X)

                margins += y * scores
            except Exception as e:
                warnings.warn(f"Ошибка при расчете отступа для {name}: {e}")
                continue

        return margins

    def _calculate_Q(self, margins: np.ndarray) -> int:
        """
        Расчет функционала качества композиции

        Parameters:
        margins: отступы объектов

        Returns:
        Q: количество объектов с отступом < 0
        """
        return np.sum(margins < 0)

    def _find_best_algorithm(self, X: np.ndarray, y: np.ndarray,
                            current_algorithms: List[Tuple[str, object]]) -> Tuple[object, str, float]:
        """
        Поиск лучшего базового алгоритма

        Parameters:
        X: признаки
        y: метки
        current_algorithms: текущий список алгоритмов

        Returns:
        best_algo: лучший алгоритм
        best_name: название лучшего алгоритма
        best_Q: лучшее значение Q
        """
        base_algos = self._create_base_algorithms()
        best_Q = float('inf')
        best_algo = None
        best_name = None

        for name, algo_class in base_algos:
            try:
                # Обучаем алгоритм
                algo = algo_class.fit(X, y)

                # Временно добавляем к текущим алгоритмам
                temp_algorithms = current_algorithms + [(name, algo)]

                # Рассчитываем отступы
                margins = self._calculate_margin(X, y, temp_algorithms)

                # Рассчитываем Q
                Q = self._calculate_Q(margins)

                if Q < best_Q:
                    best_Q = Q
                    best_algo = algo
                    best_name = name

            except Exception as e:
                warnings.warn(f"Алгоритм {name} вызвал ошибку: {e}")
                continue

        return best_algo, best_name, best_Q

    def fit(self, X: np.ndarray, y: np.ndarray,
            sample_size: float = None, stratify: bool = True) -> 'BoostingAlgorithm':
        """
        Обучение алгоритма

        Parameters:
        X: признаки
        y: метки (-1 или +1)
        sample_size: доля выборки для уменьшения (None - использовать всю)
        stratify: использовать стратификацию при уменьшении выборки

        Returns:
        self: обученный алгоритм
        """
        # Преобразование в numpy массивы
        X, y = self._ensure_numpy(X, y)

        # Уменьшение выборки если нужно
        if sample_size is not None and sample_size < 1.0:
            if stratify:
                X, _, y, _ = train_test_split(
                    X, y,
                    train_size=sample_size,
                    stratify=y,
                    random_state=self.random_state
                )
            else:
                indices = np.random.choice(
                    len(X),
                    size=int(len(X) * sample_size),
                    replace=False,
                    random_state=self.random_state
                )
                X = X[indices]
                y = y[indices]

        # Проверка меток
        unique_labels = np.unique(y)
        if not (set(unique_labels) == {-1, 1} or set(unique_labels) == {0, 1}):
            raise ValueError("Метки должны быть -1/+1 или 0/1")

        # Преобразование меток 0/1 в -1/+1 если нужно
        if set(unique_labels) == {0, 1}:
            y = np.where(y == 0, -1, 1)

        print(np.count_nonzero(y))
        print(np.count_nonzero(y==1))
        # Шаг 1: построение первого базового алгоритма
        print("Шаг 1: Построение первого базового алгоритма")
        best_algo, best_name, best_Q = self._find_best_algorithm(X, y, [])

        if best_algo is None:
            raise ValueError("Не удалось обучить ни один базовый алгоритм")

        self.base_algorithms = [(best_name, best_algo)]
        current_margins = self._calculate_margin(X, y, self.base_algorithms)

        # Сортировка по отступам
        sorted_indices = np.argsort(current_margins)
        X_sorted = X[sorted_indices]
        y_sorted = y[sorted_indices]
        margins_sorted = current_margins[sorted_indices]

        self.Q_history.append(best_Q)
        self.margins_history.append(current_margins.copy())

        print(f"Первый алгоритм: {best_name}, Q = {best_Q}")

        # Основной цикл
        for t in range(self.max_iter):
            print(f"\nИтерация {t + 1}")

            best_iter_algo = None
            best_iter_name = None
            best_iter_Q = float('inf')
            best_k = None

            # Вложенный цикл по k
            for k in range(self.l1, self.l2 + 1, self.delta_l):
                if k <= self.l0 or k >= len(X_sorted):
                    continue

                # Выбираем подвыборку от l0 до k
                sub_indices = range(self.l0, k)
                X_sub = X_sorted[sub_indices]
                y_sub = y_sorted[sub_indices]

                print(np.count_nonzero(y_sub))
                print(np.count_nonzero(y_sub==1))

                # Строим базовый алгоритм
                algo, name, Q = self._find_best_algorithm(X_sub, y_sub, self.base_algorithms)

                if Q < best_iter_Q and algo is not None:
                    best_iter_Q = Q
                    best_iter_algo = algo
                    best_iter_name = name
                    best_k = k

            # Проверяем улучшение
            if best_iter_algo is None:
                print("Не удалось найти подходящий алгоритм на этой итерации")
                break

            # Добавляем лучший алгоритм в композицию
            self.base_algorithms.append((best_iter_name, best_iter_algo))

            # Пересчитываем отступы
            current_margins = self._calculate_margin(X, y, self.base_algorithms)

            # Сортировка по отступам
            sorted_indices = np.argsort(current_margins)
            X_sorted = X[sorted_indices]
            y_sorted = y[sorted_indices]
            margins_sorted = current_margins[sorted_indices]

            # Сохраняем историю
            self.Q_history.append(best_iter_Q)
            self.margins_history.append(current_margins.copy())

            print(f"Добавлен алгоритм: {best_iter_name}, k = {best_k}, Q = {best_iter_Q}")

            # Проверяем условие остановки
            if len(self.Q_history) >= 2:
                improvement = self.Q_history[-2] - self.Q_history[-1]
                if improvement < self.min_improvement:
                    print(f"Улучшение ({improvement}) меньше минимального ({self.min_improvement}), остановка")
                    break


        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Предсказание для новых данных

        Parameters:
        X: признаки

        Returns:
        predictions: предсказанные метки (-1 или +1)
        """
        if not self.base_algorithms:
            raise ValueError("Модель не обучена")

        X, _ = self._ensure_numpy(X, None)
        margins = self._calculate_margin(X, np.ones(len(X)), self.base_algorithms)
        return np.where(margins >= 0, 1, -1)

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """
        Вероятности принадлежности классу +1

        Parameters:
        X: признаки

        Returns:
        probabilities: вероятности класса +1
        """
        if not self.base_algorithms:
            raise ValueError("Модель не обучена")

        X, _ = self._ensure_numpy(X, None)
        margins = self._calculate_margin(X, np.ones(len(X)), self.base_algorithms)
        # Преобразуем отступы в вероятности с помощью сигмоиды
        probabilities = 1 / (1 + np.exp(-margins))
        return probabilities

    def get_algorithm_info(self) -> List[Tuple[str, str]]:
        """
        Получение информации об алгоритмах в композиции

        Returns:
        info: список кортежей (уникальное имя, тип алгоритма)
        """
        info = []
        for i, (name, algo) in enumerate(self.base_algorithms):
            info.append((f"{name}_{i}", name))
        return info

# Пример использования с исправлениями
if __name__ == "__main__":
    from sklearn.datasets import make_classification

    # Генерация тестовых данных
    X, y = scaled,  fraud_labels

    # Преобразование меток в -1/+1
    y = np.where(y == 0, -1, 1)

    # Разделение на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.05, random_state=42, stratify=y
    )

    # Инициализация и обучение алгоритма
    booster = BoostingAlgorithm(
        l0=0,      # начальный индекс отсечения
        l1=45000,      # начальное значение k
        l2=55000,     # конечное значение k
        delta_l=2000, # шаг для k
        max_iter=5,  # уменьшим для теста
        min_improvement=1,
        random_state=42
    )

    try:
        # Обучение с уменьшением выборки (10%) и стратификацией
        booster.fit(X_train, y_train, sample_size=0.1, stratify=True)

        # Предсказание
        y_pred = booster.predict(X_test)

        # Оценка качества
        print("\n" + "="*50)
        print("Результаты классификации:")
        print(classification_report(y_test, y_pred, target_names=['Class -1', 'Class +1']))

        print(f"Точность: {accuracy_score(y_test, y_pred):.4f}")
        print(f"Финальное значение Q: {booster.Q_history[-1]}")
        print(f"Количество алгоритмов в композиции: {len(booster.base_algorithms)}")

        # История значений Q
        print("\nИстория значений Q:")
        for i, q in enumerate(booster.Q_history):
            print(f"Итерация {i}: Q = {q}")

    except Exception as e:
        print(f"Произошла ошибка: {e}")
        import traceback
        traceback.print_exc()

56491
684
Шаг 1: Построение первого базового алгоритма
Первый алгоритм: SVM, Q = 55807

Итерация 1
45000
0


/tmp/ipython-input-732482010.py:140: UserWarning: Алгоритм SVM вызвал ошибку: The number of classes has to be greater than one; got 1 class
  warnings.warn(f"Алгоритм {name} вызвал ошибку: {e}")
/tmp/ipython-input-732482010.py:140: UserWarning: Алгоритм LogisticRegression вызвал ошибку: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(-1)
  warnings.warn(f"Алгоритм {name} вызвал ошибку: {e}")
/tmp/ipython-input-732482010.py:140: UserWarning: Алгоритм SVM вызвал ошибку: The number of classes has to be greater than one; got 1 class
  warnings.warn(f"Алгоритм {name} вызвал ошибку: {e}")


47000
0


/tmp/ipython-input-732482010.py:140: UserWarning: Алгоритм LogisticRegression вызвал ошибку: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(-1)
  warnings.warn(f"Алгоритм {name} вызвал ошибку: {e}")
/tmp/ipython-input-732482010.py:140: UserWarning: Алгоритм SVM вызвал ошибку: The number of classes has to be greater than one; got 1 class
  warnings.warn(f"Алгоритм {name} вызвал ошибку: {e}")


49000
0


...

/tmp/ipython-input-732482010.py:140: UserWarning: Алгоритм LogisticRegression вызвал ошибку: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(-1)
  warnings.warn(f"Алгоритм {name} вызвал ошибку: {e}")
/tmp/ipython-input-732482010.py:140: UserWarning: Алгоритм SVM вызвал ошибку: The number of classes has to be greater than one; got 1 class
  warnings.warn(f"Алгоритм {name} вызвал ошибку: {e}")


55000
0


/tmp/ipython-input-732482010.py:140: UserWarning: Алгоритм LogisticRegression вызвал ошибку: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(-1)
  warnings.warn(f"Алгоритм {name} вызвал ошибку: {e}")


Добавлен алгоритм: DecisionTree, k = 45000, Q = 45000
Улучшение (0) меньше минимального (1), остановка

Результаты классификации:
              precision    recall  f1-score   support

    Class -1       0.00      0.00      0.00     29373
    Class +1       0.01      1.00      0.02       360

    accuracy                           0.01     29733
   macro avg       0.01      0.50      0.01     29733
weighted avg       0.00      0.01      0.00     29733

Точность: 0.0121
Финальное значение Q: 45000
Количество алгоритмов в композиции: 3

История значений Q:
Итерация 0: Q = 55807
Итерация 1: Q = 45000
Итерация 2: Q = 45000


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.utils import resample
from typing import List, Tuple, Optional

class SVMEnsemble:
    def __init__(self,
                 T: int = 10,
                 l0: int = 0,
                 l1: int = 10,
                 l2: int = 100,
                 delta_l: int = 10,
                 min_improvement: float = 0.01,
                 max_iter: int = 100,
                 svm_params: Optional[dict] = None):
        """
        Инициализация ансамбля SVM

        Args:
            T: Количество итераций внешнего цикла
            l0, l1, l2: параметры отсечения для выбора подвыборки
            delta_l: шаг для перебора k
            min_improvement: минимальное улучшение Q для продолжения алгоритма
            max_iter: максимальное количество итераций основного цикла
            svm_params: параметры для SVM
        """
        self.T = T
        self.l0 = l0
        self.l1 = l1
        self.l2 = l2
        self.delta_l = delta_l
        self.min_improvement = min_improvement
        self.max_iter = max_iter
        self.svm_params = svm_params or {'kernel': 'linear', 'C': 1.0}

        self.base_algorithms = []
        self.Q_history = []

    def calculate_margin(self, X: np.ndarray, y: np.ndarray) -> np.ndarray:
        """
        Вычисление отступов для всех объектов

        Args:
            X: матрица признаков
            y: метки объектов (-1 или +1)

        Returns:
            margins: вектор отступов
        """
        margins = np.zeros(len(X))
        for svm in self.base_algorithms:
            predictions = svm.predict(X)
            margins += y * predictions
        return margins

    def calculate_Q(self, margins: np.ndarray) -> int:
        """
        Вычисление функционала качества Q

        Args:
            margins: вектор отступов

        Returns:
            Q: количество объектов с отступом < 0
        """
        return np.sum(margins < 0)

    def stratified_reduce_sample(self, X: np.ndarray, y: np.ndarray,
                               target_size: int) -> Tuple[np.ndarray, np.ndarray]:
        """
        Стратифицированное уменьшение выборки

        Args:
            X: исходная матрица признаков
            y: исходные метки
            target_size: целевой размер выборки

        Returns:
            X_reduced, y_reduced: уменьшенная выборка
        """
        if target_size >= len(X):
            return X.copy(), y.copy()

        # Определяем классы
        classes = np.unique(y)
        n_per_class = target_size // len(classes)

        X_reduced_list = []
        y_reduced_list = []

        for cls in classes:
            X_cls = X[y == cls]
            y_cls = y[y == cls]

            # Выбираем случайную подвыборку для каждого класса
            if len(X_cls) > n_per_class:
                X_sampled, y_sampled = resample(X_cls, y_cls,
                                              n_samples=n_per_class,
                                              random_state=42)
            else:
                X_sampled, y_sampled = X_cls, y_cls

            X_reduced_list.append(X_sampled)
            y_reduced_list.append(y_sampled)

        X_reduced = np.vstack(X_reduced_list)
        y_reduced = np.hstack(y_reduced_list)

        return X_reduced, y_reduced

    def fit(self, X: np.ndarray, y: np.ndarray,
           reduce_sample: bool = False, target_size: Optional[int] = None) -> 'SVMEnsemble':
        """
        Обучение ансамбля

        Args:
            X: матрица признаков
            y: метки объектов (-1 или +1)
            reduce_sample: флаг стратифицированного уменьшения выборки
            target_size: целевой размер уменьшенной выборки

        Returns:
            self: обученная модель
        """
        # Стратифицированное уменьшение выборки при необходимости
        if reduce_sample and target_size is not None:
            X, y = self.stratified_reduce_sample(X, y, target_size)

        # Инициализация
        n_samples = len(X)
        margins = np.zeros(n_samples)

        # Первый базовый алгоритм
        print("Строится первый базовый алгоритм...")
        svm1 = SVC(**self.svm_params)
        svm1.fit(X, y)
        self.base_algorithms.append(svm1)

        # Пересчет отступов и сортировка
        margins = self.calculate_margin(X, y)
        Q_current = self.calculate_Q(margins)
        self.Q_history.append(Q_current)

        print(f"Начальный Q: {Q_current}")

        # Основной цикл
        iteration = 0
        improvement = float('inf')

        while improvement > self.min_improvement and iteration < self.max_iter:
            iteration += 1
            print(f"\n--- Итерация {iteration} ---")

            best_svm = None
            best_Q = float('inf')
            best_margins = None
            best_indices = None

            # Внешний цикл на T итераций
            for t in range(self.T):
                print(f"  Внешняя итерация {t+1}/{self.T}")

                # Сортировка по отступам
                sorted_indices = np.argsort(margins)
                X_sorted = X[sorted_indices]
                y_sorted = y[sorted_indices]

                # Внутренний цикл по k
                for k in range(self.l1, min(self.l2, n_samples) + 1, self.delta_l):
                    # Выбор подвыборки от l0 до k
                    start_idx = max(0, self.l0)
                    end_idx = min(k, n_samples)

                    if end_idx <= start_idx:
                        continue

                    X_subset = X_sorted[start_idx:end_idx]
                    y_subset = y_sorted[start_idx:end_idx]

                    # Обучение SVM на подвыборке
                    svm_candidate = SVC(**self.svm_params)
                    svm_candidate.fit(X_subset, y_subset)

                    # Временное добавление кандидата для оценки Q
                    temp_algorithms = self.base_algorithms + [svm_candidate]

                    # Вычисление отступов с временным алгоритмом
                    temp_margins = margins.copy()
                    predictions = svm_candidate.predict(X)
                    temp_margins += y * predictions

                    # Вычисление Q
                    Q_candidate = self.calculate_Q(temp_margins)
                    best_svm = svm_candidate
                    # Проверка на лучший алгоритм
                    if Q_candidate < best_Q:
                        best_Q = Q_candidate
                        best_svm = svm_candidate
                        best_margins = temp_margins.copy()
                        best_indices = (start_idx, end_idx)

            # Добавление лучшего алгоритма в ансамбль
            if best_svm is not None:
                self.base_algorithms.append(best_svm)
                margins = best_margins
                Q_previous = Q_current
                Q_current = best_Q
                improvement = (Q_previous - Q_current) / Q_previous if Q_previous > 0 else 0

                self.Q_history.append(Q_current)
                print(f"  Лучший алгоритм: подвыборка [{best_indices[0]}, {best_indices[1]})")
                print(f"  Q: {Q_current} (улучшение: {improvement:.4f})")
            else:
                print("  Не найдено подходящего алгоритма")
                break

        print(f"\nОбучение завершено. Количество базовых алгоритмов: {len(self.base_algorithms)}")
        print(f"Финальный Q: {Q_current}")

        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Предсказание для новых данных

        Args:
            X: матрица признаков

        Returns:
            predictions: предсказанные метки (-1 или +1)
        """
        if not self.base_algorithms:
            raise ValueError("Модель не обучена")

        margins = np.zeros(len(X))
        for svm in self.base_algorithms:
            predictions = svm.predict(X)
            margins += predictions

        return np.sign(margins)

    def predict_margins(self, X: np.ndarray) -> np.ndarray:
        """
        Вычисление отступов для новых данных

        Args:
            X: матрица признаков

        Returns:
            margins: вектор отступов
        """
        if not self.base_algorithms:
            raise ValueError("Модель не обучена")

        margins = np.zeros(len(X))
        for svm in self.base_algorithms:
            predictions = svm.predict(X)
            margins += predictions

        return margins

# Пример использования
if __name__ == "__main__":
    from sklearn.datasets import make_classification
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score

    # Генерация тестовых данных
    X, y = scaled,  fraud_labels
    y = np.where(y == 0, -1, 1)  # Преобразование в метки -1/+1

    # Разделение на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05,
                                                        random_state=42, stratify=y)

    # Создание и обучение модели
    model = SVMEnsemble(
        T=5,
        l0=200,
        l1=40000,
        l2=50000,
        delta_l=2000,
        min_improvement=0.01,
        max_iter=10,
        svm_params={'kernel': 'linear', 'C': 1.0}
    )

    # Обучение с стратифицированным уменьшением выборки
    model.fit(X_train, y_train, reduce_sample=True, target_size=50000)

    # Предсказание
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nТочность на тестовой выборке: {accuracy:.4f}")

    # История изменения Q
    print(f"\nИстория Q: {model.Q_history}")

Строится первый базовый алгоритм...
Начальный Q: 901

--- Итерация 1 ---
  Внешняя итерация 1/5
  Внешняя итерация 2/5
  Внешняя итерация 3/5
  Внешняя итерация 4/5
  Внешняя итерация 5/5
  Не найдено подходящего алгоритма

Обучение завершено. Количество базовых алгоритмов: 1
Финальный Q: 901


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(



Точность на тестовой выборке: 0.9837

История Q: [np.int64(901)]


In [ ]:
# import numpy as np
# from sklearn.svm import SVC
# from sklearn.model_selection import train_test_split
# from typing import List, Tuple, Optional

# def stratified_subsample(X: np.ndarray, y: np.ndarray, frac: float = 1.0, random_state: Optional[int] = None):
#     """
#     Стратифицированное уменьшение выборки.
#     Если frac == 1.0 — возвращает исходные данные.
#     """
#     if frac >= 1.0:
#         return X, y
#     _, X_sub, _, y_sub = train_test_split(
#         X, y, test_size=frac, stratify=y, random_state=random_state
#     )
#     return X_sub, y_sub


# def compute_margin(y_true: np.ndarray, predictions: np.ndarray) -> np.ndarray:
#     """
#     Вычисляет отступы: M_i = y_i * sum_b f_b(x_i)
#     Здесь predictions: (n_samples, n_base_algos) — результаты всех базовых алгоритмов.
#     """
#     return y_true * np.sum(predictions, axis=1)


# def compute_Q(y_true: np.ndarray, predictions: np.ndarray) -> int:
#     """
#     Функционал качества: количество объектов с отступом < 0.
#     """
#     margins = compute_margin(y_true, predictions)
#     return np.sum(margins < 0)


# def fit_svm(X_train: np.ndarray, y_train: np.ndarray, **svm_kwargs) -> SVC:
#     """
#     Обучает SVM на подвыборке.
#     """
#     clf = SVC(kernel='linear', **svm_kwargs)
#     clf.fit(X_train, y_train)
#     return clf


# def predict_with_ensemble(ensemble: List[SVC], X: np.ndarray) -> np.ndarray:
#     """
#     Возвращает предсказания всех моделей в ансамбле: shape (n_samples, len(ensemble))
#     """
#     if not ensemble:
#         return np.zeros((X.shape[0], 0))
#     preds = np.column_stack([clf.predict(X) for clf in ensemble])
#     return preds


# def margin_based_boosting(
#     X: np.ndarray,
#     y: np.ndarray,
#     T: int = 10,
#     l0: int = 0,
#     l1: int = 10,
#     l2: int = 100,
#     delta_l: int = 10,
#     svm_kwargs: dict = None,
#     frac_subsample: float = 1.0,
#     random_state: Optional[int] = None,
#     min_Q_improvement: float = 1.0,
# ) -> List[SVC]:
#     """
#     Реализация описанного margin-based boosting алгоритма.

#     Параметры:
#     - X, y: обучающая выборка (y ∈ {-1, +1})
#     - T: макс. число итераций
#     - l0, l1, l2, delta_l: параметры отсечения индексов
#     - svm_kwargs: параметры SVM
#     - frac_subsample: доля выборки после стратифицированного уменьшения
#     - min_Q_improvement: минимальное улучшение Q для продолжения
#     """
#     if svm_kwargs is None:
#         svm_kwargs = {}

#     # Стратифицированное уменьшение выборки
#     X, y = stratified_subsample(X, y, frac=frac_subsample, random_state=random_state)
#     n_samples = X.shape[0]

#     # Проверка корректности индексов
#     l0 = max(0, l0)
#     l1 = max(l0 + 1, l1)
#     l2 = min(n_samples, l2)
#     if l1 >= l2:
#         raise ValueError("l1 must be < l2 and within dataset size")

#     # Инициализация ансамбля
#     ensemble: List[SVC] = []

#     # Инициализируем предсказания (пока пусто)
#     all_preds = predict_with_ensemble(ensemble, X)  # shape (n, 0)
#     margins = compute_margin(y, all_preds)  # все нули на старте
#     sorted_indices = np.argsort(margins)

#     Q_prev = compute_Q(y, all_preds)
#     Q_prev=1000000

#     for t in range(T):
#         best_clf = None
#         best_Q = float('inf')
#         best_preds = None

#         # Вложенный цикл по k
#         for k in range(l1, l2 + 1, delta_l):
#             # Подвыборка: от l0 до k (в отсортированной выборке)
#             idxs = sorted_indices[l0:k]
#             if len(idxs) == 0:
#                 continue

#             X_sub = X[idxs]
#             y_sub = y[idxs]

#             # Обучаем SVM
#             try:
#                 clf = fit_svm(X_sub, y_sub, **svm_kwargs)
#             except ValueError:
#                 # Пропускаем, если SVM не смог обучиться (например, один класс)
#                 continue

#             # Добавляем предсказание этой модели ко всем предыдущим
#             new_preds = np.column_stack([all_preds, clf.predict(X)]) if ensemble else clf.predict(X).reshape(-1, 1)

#             Q_candidate = compute_Q(y, new_preds)

#             if Q_candidate < best_Q:
#                 best_Q = Q_candidate
#                 best_clf = clf
#                 best_preds = new_preds

#         # Если не нашли лучшего — выходим
#         if best_clf is None:
#             print(f"Итерация {t}: не удалось обучить ни один базовый алгоритм. Завершение.")
#             break

#         # Добавляем лучший алгоритм в ансамбль
#         ensemble.append(best_clf)
#         all_preds = best_preds

#         # Пересчитываем отступы и сортируем
#         margins = compute_margin(y, all_preds)
#         sorted_indices = np.argsort(margins)

#         # Критерий останова: Q перестал существенно уменьшаться
#         Q_curr = best_Q
#         # if Q_prev - Q_curr < min_Q_improvement:
#         #     print(f"Итерация {t}: улучшение Q = {Q_prev - Q_curr:.2f} < {min_Q_improvement}. Завершение.")
#         #     break
#         Q_prev = Q_curr

#         print(f"Итерация {t+1}/{T}: Q = {Q_curr}")

#     return ensemble


# # Пример использования:
# if __name__ == "__main__":
#     from sklearn.datasets import make_classification

#     # Генерируем данные с метками ±1
#     X, y = scaled.to_numpy(),  fraud_labels.to_numpy()
#     y = np.where(y == 0, -1, 1)  # Преобразуем 0/1 → -1/+1

#     # Запуск алгоритма
#     ensemble = margin_based_boosting(
#         X, y,
#         T=10,
#         l0=50,
#         l1=40000,
#         l2=50000,
#         delta_l=2000,
#         frac_subsample=0.1,  # уменьшаем выборку до 10%
#         random_state=42,
#         min_Q_improvement=1.0
#     )

#     print(f"\nОбучено {len(ensemble)} моделей.")

In [ ]:
scaled.to_numpy()

array([[ 1.50433671,  0.95132111, -0.91895216, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.50433671, -0.19030159, -0.91895216, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.50433671,  0.16417774, -0.91895216, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.75923026,  0.22135761, -1.23379506, ...,  0.        ,
         0.        ,  0.        ],
       [-0.75923026, -0.10709006, -1.23379506, ...,  1.        ,
         0.        ,  0.        ],
       [-0.75923026, -0.04425503, -1.23379506, ...,  1.        ,
         0.        ,  0.        ]])

# Комитетный бустинг

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

import numpy as np
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.exceptions import ConvergenceWarning
from typing import List, Tuple, Optional, Any
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)


def stratified_subsample(X: np.ndarray, y: np.ndarray, frac: float = 1.0, random_state: Optional[int] = None):
    if frac >= 1.0:
        return X, y
    _, X_sub, _, y_sub = train_test_split(
        X, y, test_size=frac, stratify=y, random_state=random_state
    )
    return X_sub, y_sub


def compute_margin(y_true: np.ndarray, predictions: np.ndarray) -> np.ndarray:
    return y_true * np.sum(predictions, axis=1)


def compute_Q(y_true: np.ndarray, predictions: np.ndarray) -> int:
    margins = compute_margin(y_true, predictions)
    return np.sum(margins < 0)


def predict_with_ensemble(ensemble: List[Any], X: np.ndarray) -> np.ndarray:
    if not ensemble:
        return np.zeros((X.shape[0], 0))
    preds = np.column_stack([clf.predict(X) for clf in ensemble])
    return preds


def fit_candidate_models(X_train: np.ndarray, y_train: np.ndarray, random_state: Optional[int] = None):
    """
    Обучает три модели: SVM, дерево, лог. регрессия.
    Возвращает список (модель, название).
    """
    candidates = []

    # SVM с линейным ядром
    try:
        svm = SVC(kernel='linear', random_state=random_state)
        svm.fit(X_train, y_train)
        candidates.append(svm)
    except Exception:
        pass

    # Дерево решений
    try:
        tree = DecisionTreeClassifier(random_state=random_state, max_depth=10)
        tree.fit(X_train, y_train)
        candidates.append(tree)
    except Exception:
        pass

    # Логистическая регрессия (метки должны быть 0/1 для sklearn, но clf сам преобразует -1/+1)
    try:
        lr = LogisticRegression(random_state=random_state, max_iter=1000)
        lr.fit(X_train, y_train)
        candidates.append(lr)
    except Exception:
        pass

    return candidates


def margin_based_boosting_multimodel(
    X: np.ndarray,
    y: np.ndarray,
    T: int = 10,
    l0: int = 0,
    l1: int = 10,
    l2: int = 100,
    delta_l: int = 10,
    frac_subsample: float = 1.0,
    random_state: Optional[int] = None,
    min_Q_improvement: float = 1.0,
) -> List[Any]:
    """
    Margin-based boosting с выбором лучшей модели из SVM, дерева и лог. регрессии.
    """
    # Стратифицированное уменьшение
    X, y = stratified_subsample(X, y, frac=frac_subsample, random_state=random_state)
    n_samples = X.shape[0]

    l0 = max(0, l0)
    l1 = max(l0 + 1, l1)
    l2 = min(n_samples, l2)
    if l1 >= l2:
        raise ValueError("l1 must be < l2 and within dataset size")

    ensemble: List[Any] = []
    all_preds = predict_with_ensemble(ensemble, X)  # (n, 0)
    margins = compute_margin(y, all_preds)
    sorted_indices = np.argsort(margins)
    Q_prev = compute_Q(y, all_preds)
    Q_prev=100000

    for t in range(T):
        best_candidate = None
        best_Q = float('inf')
        best_preds = None

        # Перебираем k
        for k in range(l1, l2 + 1, delta_l):
            idxs = sorted_indices[l0:k]
            if len(idxs) == 0:
                continue

            X_sub = X[idxs]
            y_sub = y[idxs]

            # Обучаем три модели
            candidates = fit_candidate_models(X_sub, y_sub, random_state=random_state)

            for clf in candidates:
                try:
                    new_pred = clf.predict(X)
                except Exception:
                    continue

                # Добавляем к текущим предсказаниям
                if ensemble:
                    candidate_preds = np.column_stack([all_preds, new_pred])
                else:
                    candidate_preds = new_pred.reshape(-1, 1)

                Q_candidate = compute_Q(y, candidate_preds)

                if Q_candidate < best_Q:
                    best_Q = Q_candidate
                    best_candidate = clf
                    numforprint=candidates.index(clf)
                    best_preds = candidate_preds

        if best_candidate is None:
            print(f"Итерация {t}: не удалось обучить ни один кандидат. Завершение.")
            break
        if numforprint==0:
          print("Добавлен базовый алгоритм: SVC")
        elif numforprint==1:
          print("Добавлен базовый алгоритм: Дерево решений")
        elif numforprint==2:
          print("Добавлен базовый алгоритм: Логистическая регрессия")


        # Добавляем лучшую модель
        ensemble.append(best_candidate)
        all_preds = best_preds

        # Обновляем отступы и сортировку
        margins = compute_margin(y, all_preds)
        sorted_indices = np.argsort(margins)


        Q_curr = best_Q
        print(Q_curr, Q_prev)
        if Q_prev - Q_curr < min_Q_improvement:
            print(f"Итерация {t+1}: улучшение Q = {Q_prev - Q_curr:.2f} < {min_Q_improvement}. Завершение.")
            # break
        Q_prev = Q_curr

        print(f"Итерация {t+1}/{T}: Q = {Q_curr}")

    return ensemble


# Пример использования:
if __name__ == "__main__":
    from sklearn.datasets import make_classification

    X, y = scaled.to_numpy(),  fraud_labels.to_numpy()
    y = np.where(y == 0, -1, 1)  # Преобразуем в -1 / +1

    ensemble = margin_based_boosting_multimodel(
        X, y,
        T=10,
        l0=50,
        l1=40000,
        l2=50000,
        delta_l=2000,
        frac_subsample=0.1,  # уменьшаем выборку до 10%
        random_state=42,
        min_Q_improvement=1.0
    )

    print(f"\nОбучено {len(ensemble)} моделей.")

Добавлен базовый алгоритм: Дерево решений
137 100000
Итерация 1/10: Q = 137
Добавлен базовый алгоритм: Дерево решений
93 137
Итерация 2/10: Q = 93
Добавлен базовый алгоритм: Дерево решений
109 93
Итерация 3: улучшение Q = -16.00 < 1.0. Завершение.
Итерация 3/10: Q = 109
Добавлен базовый алгоритм: Дерево решений
96 109
Итерация 4/10: Q = 96
Добавлен базовый алгоритм: Дерево решений
102 96
Итерация 5: улучшение Q = -6.00 < 1.0. Завершение.
Итерация 5/10: Q = 102
Добавлен базовый алгоритм: Дерево решений
96 102
Итерация 6/10: Q = 96
Добавлен базовый алгоритм: Дерево решений
107 96
Итерация 7: улучшение Q = -11.00 < 1.0. Завершение.
Итерация 7/10: Q = 107
Добавлен базовый алгоритм: Логистическая регрессия
101 107
Итерация 8/10: Q = 101
Добавлен базовый алгоритм: Дерево решений
107 101
Итерация 9: улучшение Q = -6.00 < 1.0. Завершение.
Итерация 9/10: Q = 107
Добавлен базовый алгоритм: Логистическая регрессия
100 107
Итерация 10/10: Q = 100

Обучено 10 моделей.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# -------------------------------
# 1. Данные (замените на свои при необходимости)
# -------------------------------
X, y = scaled.to_numpy(),  fraud_labels.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)

# -------------------------------
# 2. Модели
# -------------------------------
models = {
    'SVM': SVC(probability=True, random_state=42),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'LogisticRegression': LogisticRegression(max_iter=1000, random_state=42)
}

# -------------------------------
# 3. Обучение и получение предсказаний НА ТЕСТОВОЙ ВЫБОРКЕ
# -------------------------------
test_predictions = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    test_predictions[name] = model.predict(X_test)

# -------------------------------
# 4. Оценка p как доля ошибок на тестовой выборке
# -------------------------------
errors = {}
for name, y_pred in test_predictions.items():
    num_errors = np.sum(y_pred != y_test)
    p = num_errors / len(y_test)
    errors[name] = p

# -------------------------------
# 5. Расчёт весов: w = ln((1 - p) / p)
# -------------------------------
weights = {}
eps = 1e-10  # защита от деления на 0 и log(0)
for name, p in errors.items():
    p = np.clip(p, eps, 1 - eps)
    w = np.log((1 - p) / p)
    weights[name] = w

print("Вероятности ошибки (p):")
for name, p in errors.items():
    print(f"{name}: {p:.4f}")

print("\nВеса моделей:")
for name, w in weights.items():
    print(f"{name}: {w:.4f}")

# -------------------------------
# 6. Построение композиции на тестовой выборке
# -------------------------------
weighted_sum = np.zeros(len(X_test))
for name in models.keys():
    preds = test_predictions[name]
    # Преобразуем метки в -1 / +1 для корректного взвешивания
    votes = np.where(preds == 1, 1, -1)
    weighted_sum += weights[name] * votes

y_pred_composite = (weighted_sum > 0).astype(int)

# -------------------------------
# 7. Сравнение метрик
# -------------------------------
results = {}

# Метрики базовых моделей
for name, y_pred in test_predictions.items():
    results[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred)
    }

# Метрика композиции
results['Composite'] = {
    'Accuracy': accuracy_score(y_test, y_pred_composite),
    'Precision': precision_score(y_test, y_pred_composite),
    'Recall': recall_score(y_test, y_pred_composite),
    'F1': f1_score(y_test, y_pred_composite)
}

# -------------------------------
# 8. Вывод
# -------------------------------
df_results = pd.DataFrame(results).T
print("\nСравнение моделей (метрики на тестовой выборке):")
print(df_results.round(4))

Вероятности ошибки (p):
SVM: 0.0045
DecisionTree: 0.0068
LogisticRegression: 0.0046

Веса моделей:
SVM: 5.4052
DecisionTree: 4.9849
LogisticRegression: 5.3803

Сравнение моделей (метрики на тестовой выборке):
                    Accuracy  Precision  Recall      F1
SVM                   0.9955     0.9068  0.7028  0.7919
DecisionTree          0.9932     0.7090  0.7444  0.7263
LogisticRegression    0.9954     0.8933  0.7056  0.7884
Composite             0.9955     0.8963  0.7120  0.7936
